In [1]:
import os, pandas

BASE_DIR = os.getcwd() # Ou spécifiez un chemin absolu
DATA_FOLDER = os.path.join(BASE_DIR, "data", "BULLETINS")
OUTPUT_FOLDER = os.path.join(BASE_DIR, "output")

# TD2
XML_INITIAL_FILE = os.path.join(OUTPUT_FOLDER, "corpus_initial.xml")

# TD3
XML_FILTRE_FILE = os.path.join(OUTPUT_FOLDER, "corpus_filtre.xml")
SEGMENTATION_FILE = os.path.join(OUTPUT_FOLDER, "segmentation.tsv")
TF_FILE = os.path.join(OUTPUT_FOLDER, "tf.tsv")
IDF_FILE = os.path.join(OUTPUT_FOLDER, "idf.tsv")
TFIDF_FILE = os.path.join(OUTPUT_FOLDER, "tfidf.tsv")
ANTI_DICT_FILE = os.path.join(OUTPUT_FOLDER, "anti_dictionnaire.txt")

# TD4
FILTERED_XML_FILE = os.path.join(OUTPUT_FOLDER, "corpus_filtre.xml")
INDEX_OUTPUT_DIR = os.path.join(OUTPUT_FOLDER, "index_files")
COMPARISON_OUTPUT_DIR = os.path.join(OUTPUT_FOLDER, "lemma_comparison")

STEMMED_REPLACEMENT = os.path.join(OUTPUT_FOLDER, "stemmed_replacement.tsv")
INDEX_FILE = os.path.join(INDEX_OUTPUT_DIR, "index.xml")

## Chargement des documents bruts dans la classe Corpus (conteneur)

In [2]:
from index import BS4Parser, Corpus

documents = BS4Parser().process_folder(DATA_FOLDER, limit=20)  # Process all files inside the folder
Corpus(documents = documents)

Modèle spaCy 'fr_core_news_sm' chargé.
Stemmer Snowball (français) chargé.


Corpus(documents=[Document(fichier='67068.htm', numero='258', date=datetime.datetime(2011, 6, 21, 0, 0), rubrique='Focus', titre='Physique : Mathias Fink, un bel exemple de chercheur qui innove', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact='Institut Langevin "Ondes et Images" - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr', texte='Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l\'Innovation, dont Valérie Pécresse, ministre de l\'Enseignement Supérieur et de la Recherche est à l\'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l\'économiste Esther Duflo, le roboticien François Pierrot et le physi

## Sauvegarde au format XML, rechargement (debug)

In [3]:
STORAGE_TAGS = {"Corpus": "corpus", "documents": "bulletins", "Document": "bulletin", "Image": "image"}
with open(XML_INITIAL_FILE, "w", encoding="utf-8") as file:
    file.write(Corpus(documents=documents).model_dump_xml_str_pretty(tags=STORAGE_TAGS))
with open(XML_INITIAL_FILE, "r", encoding="utf-8") as file:
    CORPUS = Corpus.model_validate_xml(file.read(), tags=STORAGE_TAGS)

FILTERED_CORPUS = CORPUS.model_copy(deep=True)
CORPUS

Corpus(documents=[Document(fichier='67068.htm', numero='258', date=datetime.datetime(2011, 6, 21, 0, 0), rubrique='Focus', titre='Physique : Mathias Fink, un bel exemple de chercheur qui innove', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact='Institut Langevin "Ondes et Images" - Mathias Fink - email : mathias.fink@espci.fr - http://www.institut-langevin.espci.fr', texte='Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l\'Innovation, dont Valérie Pécresse, ministre de l\'Enseignement Supérieur et de la Recherche est à l\'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l\'économiste Esther Duflo, le roboticien François Pierrot et le physi

## Tokénisation des documents, sauvegarde des tokens

In [4]:
FILTERED_CORPUS.apply_standardization(["texte", "titre", "images.legende"])

Application de la normalisation sur les attributs spécifiés...


In [5]:
FILTERED_CORPUS.documents[18].tokens["legendes"]

{'l': 1,
 'ouvrage': 1,
 'de': 1,
 'marion': 1,
 'guillou': 1,
 'et': 1,
 'gérard': 1,
 'matheron': 1}

In [6]:
FILTERED_CORPUS.inverted_token_index_flattened().head(3)

,token,document_id
0,physique,67068.htm
1,mathias,67068.htm
2,fink,67068.htm


In [7]:
FILTERED_CORPUS.inverted_token_index_flattened().shape, FILTERED_CORPUS.inverted_token_index().to_dataframe().shape

((5508, 2), (2789, 34))

In [8]:
from index import InvertedIndex

FILTERED_CORPUS.inverted_token_index().to_dataframe().to_csv(SEGMENTATION_FILE, sep="\t", index=False, header=False, encoding="utf-8")

df = pandas.read_csv(SEGMENTATION_FILE, sep="\t", header=None)
index = InvertedIndex.from_dataframe(df)
print(len(index.keys()), "unique tokens in corpus")
df.head(4)

2789 unique tokens in corpus


,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,physique,67068.htm,67068.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mathias,67068.htm,67068.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fink,67068.htm,67068.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,un,67068.htm,67068.htm,67071.htm,67383.htm,67383.htm,67385.htm,67386.htm,67387.htm,67388.htm,...,67795.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Calcul des métriques tf/logtf

In [9]:
FILTERED_CORPUS.token_index().tfidf.head(3)

,document_id,mot,tf_idf
0,67068.htm,physique,13.0103
1,67068.htm,mathias,13.0103
2,67068.htm,fink,13.0103


## Génération et application de l'anti dictionnaire au corpus

In [10]:
with open(ANTI_DICT_FILE, "w+", encoding="utf-8") as file:
    file.writelines([f"{token}\t\"\"\n" for token in CORPUS.token_index().get_irrelevant_terms()])
anti_dictionnaire = pandas.read_csv(ANTI_DICT_FILE, sep="\t", header=None, na_filter=False)
anti_dictionnaire.head(3)

,0,1
0,au,
1,ces,
2,d,


In [11]:
FILTERED_CORPUS.apply_substitutions(["texte", "titre", "images.legende"], anti_dictionnaire)

Application des substitutions sur les attributs spécifiés...


In [12]:
print(CORPUS.documents[0].texte)
print("---")
print(FILTERED_CORPUS.documents[0].texte)

Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l'Innovation, dont Valérie Pécresse, ministre de l'Enseignement Supérieur et de la Recherche est à l'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l'économiste Esther Duflo, le roboticien François Pierrot et le physicien Mathias Fink. Directeur de l'Institut Langevin "Ondes et Images", créé en janvier 2009 au sein de l'Ecole de Physique et de Chimie Industrielles de la Ville de Paris (ESPCI), Mathias Fink est un remarquable exemple de ces chercheurs qui innovent. Les travaux de son équipe ont en effet abouti à la création de quatre start-ups, qui totalisent 230 personnes, dans des domaines aussi variés que la méd

## Expérimentation des procédés de Stemming et de Lemmatisation
(TD4)

In [15]:
from index import spacy_lemmas, snowball_stems

tokens = list(FILTERED_CORPUS.tokens().keys())

snowball_stems = snowball_stems(tokens)
spacy_lemmas = spacy_lemmas(tokens)

pandas.DataFrame({
    "Token": list(snowball_stems["word"]), 
    "Spacy": list(spacy_lemmas["stem"]), 
    "Snowball": list(snowball_stems["stem"]),
}).tail(3)

,Token,Spacy,Snowball
2721,évoque,évoqu,évoqu
2722,évoquer,évoquer,évoqu
2723,êtes,être,ête
2724,être,être,être
2725,êtres,être,être


In [16]:
pandas.DataFrame({
    "total_mots": [len(snowball_stems["stem"])] * 2,
    "racines_uniques": [x["stem"].nunique() for x in (spacy_lemmas, snowball_stems)],
    "mots_non_changes": [x[x["word"] == x["stem"]].shape[0] for x in (spacy_lemmas, snowball_stems)]
}, index=["Spacy", "Snowball"]).T

,Spacy,Snowball
total_mots,2726,2726
racines_uniques,2137,1980
mots_non_changes,1485,676


### Création d'un corpus lemmatisé

In [26]:
stems = spacy_lemmas
with open(STEMMED_REPLACEMENT, "w+", encoding="utf-8") as file:
    file.writelines([f"{word}\t\"{stem}\n" for word, stem in zip(stems['word'], stems['stem'])])
    
FILTERED_CORPUS.apply_substitutions(STEMMED_REPLACEMENT)

with open(XML_FILTRE_FILE, "w", encoding="utf-8") as file:
    file.write(FILTERED_CORPUS.model_dump_xml_str_pretty(tags=STORAGE_TAGS))

Application des substitutions depuis g:\GitHub\UTC\LO17\PremierDM\output\stemmed_replacement.tsv sur 10 documents...
  ... 10/10 documents traités.
Substitutions appliquées. 10 documents ont été modifiés.
Le cache des documents a été correctement invalidé pour les documents modifiés.


In [27]:
print(CORPUS.documents[0].corps)
print("---")
print(FILTERED_CORPUS.documents[0].corps)

Physique : Mathias Fink, un bel exemple de chercheur qui innove
Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l'Innovation, dont Valérie Pécresse, ministre de l'Enseignement Supérieur et de la Recherche est à l'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l'économiste Esther Duflo, le roboticien François Pierrot et le physicien Mathias Fink. Directeur de l'Institut Langevin "Ondes et Images", créé en janvier 2009 au sein de l'Ecole de Physique et de Chimie Industrielles de la Ville de Paris (ESPCI), Mathias Fink est un remarquable exemple de ces chercheurs qui innovent. Les travaux de son équipe ont en effet abouti à la création de quatre start-ups, qui tot

## Création de fichiers d'index inversés sur les tokens lemmatisés, pour différents champs

In [47]:
FILTERED_CORPUS.inverted_token_index(zones=["titre", "texte"]).find_docs(tokens=["trouver", "première"])

['67068.htm', '67383.htm']

In [46]:
CORPUS["67068.htm"].read_zones

{'titre': 'Physique : Mathias Fink, un bel exemple de chercheur qui innove',
 'texte': 'Le 27 avril dernier, le CNRS décernait pour la première fois la Médaille de l\'Innovation, dont Valérie Pécresse, ministre de l\'Enseignement Supérieur et de la Recherche est à l\'origine de la création. Le CNRS souhaite ainsi honorer des chercheurs et ingénieurs travaillant dans des établissements publics de recherche, des universités, des grandes écoles mais aussi des industriels qui développent des innovations marquantes. Pour cette première édition, cette nouvelle distinction a été attribuée à trois chercheurs réputés : l\'économiste Esther Duflo, le roboticien François Pierrot et le physicien Mathias Fink. Directeur de l\'Institut Langevin "Ondes et Images", créé en janvier 2009 au sein de l\'Ecole de Physique et de Chimie Industrielles de la Ville de Paris (ESPCI), Mathias Fink est un remarquable exemple de ces chercheurs qui innovent. Les travaux de son équipe ont en effet abouti à la créatio

## Correcteur
(TD5)

# TODO : Refaire le correcteur en utilisant le code existant de rarrr

In [ ]:
from index import Correcteur

lexique = list(CORPUS.tokens.keys())
CORRECTEUR = Correcteur(lexique)

CORRECTEUR.process_sentence("bonjour j'aime les dromadaires et les chameaux de Pécras", max_overflow=4)

['bon', None, 'les', None, 'et', 'les', 'champ', 'de', 'pécresse']

In [ ]:
CORRECTEUR.search_tree(Correcteur(lexique).prefix_tree, "piaep")

(['pierrot', 'pierre', 'pièce', 'pilote', 'piste', 'pistes'], 2)

## Recherche dans un Index

In [ ]:
# Construction de l'index
from typing import List, Dict

INDEX: Dict[str, Dict[str, List[str]]] = {
    "texte": {},
    "titre": {},
    "rubrique": {},
    "date": {},
}

for token in FILTERED_CORPUS.tokens.keys():
    for doc in FILTERED_CORPUS.documents:
        if token in doc.texte:
            INDEX["texte"].setdefault(token, []).append(doc.document_id)
        if token in doc.titre:
            INDEX["titre"].setdefault(token, []).append(doc.document_id)
        if token in doc.rubrique:
            INDEX["rubrique"].setdefault(token, []).append(doc.document_id)

for doc in FILTERED_CORPUS.documents:        
    INDEX["date"].setdefault(doc.date, []).append(doc.document_id)
        
list(INDEX["texte"].keys())[:5]

['physique', 'mathia', 'fink', 'bel', 'exemple']

In [ ]:
from typing import List
from index import Document

CORRECTEUR = Correcteur(lexique)
ANTI_DICT = CORPUS.anti_dict()

def tokenize(text: str) -> List[str]:
    tokens = CORRECTEUR.process_sentence(text, max_overflow=4)
    relevant_tokens = [token for token in tokens if token]
    lemmatized_tokens = FILTERED_CORPUS.spacy_lemmatize(" ".join(relevant_tokens))
    return [token for token in lemmatized_tokens if token not in ANTI_DICT]

def get_doc(doc_id: str) -> Document:
    """
    Récupère un document à partir de son ID.
    """
    for doc in CORPUS.documents:
        if doc.document_id == doc_id:
            return doc
    return None

def search(input: str) -> List[Document]:
    tokens = tokenize(input)
    print(f"Tokens: {tokens}")
    
    relevant_docs = {key: set() for key in INDEX.keys()}
    
    for field, token_mapping in INDEX.items():
        for token, doc_ids in token_mapping.items():
            if token in tokens:
                relevant_docs[field].update(doc_ids)

    return relevant_docs

search("Bonjour j'aime les carottes pourries pourrir pourra")

Tokens: ['bon', 'le', 'carbone', 'pouvoir', 'pouvoir', 'pouvoir']


{'texte': {'67068.htm',
  '67383.htm',
  '67385.htm',
  '67386.htm',
  '67387.htm',
  '67388.htm',
  '67389.htm',
  '67390.htm',
  '67391.htm'},
 'titre': {'67391.htm'},
 'rubrique': set(),
 'date': set()}

## TODO
* Fixer les appels d'anti dict chez raphael
* Fixer les appels de .tokens() chez raphael
* Fixer les appels de .corps chez raphael